# Tutorial 2 - Termodinámica Avanzada

# Cálculo de flash de dos fases (VLE y LLE)

El flash isotérmico-isobárico de dos fases se lleva a cabo mediante una combinación de sustitución sucesiva acelerada (ASS) para actualizar las composiciones de fases y el método de Halley para resolver el balance de masas de Rachford-Rice.

$$ FO = \sum_{i=1}^c \left( x_i^\beta - x_i^\alpha \right) = \sum_{i=1}^c \frac{z_i (K_i-1)}{1 +\psi (K_i-1)} $$

Donde, $K_i = x_i^\beta/x_i^\alpha =\hat{\phi}_i^\alpha /\hat{\phi}_i^\beta $ representa la constante de equilibrio y $\psi$ la fracción de la fase $\beta$. El método descrito puede ser lento a altas presiones, por esa razón, el número de ciclos de ASS se limita a ciclos ``nacc`` y si no se llega a una solución el algoritmo cambia a un procedimiento de segundo orden basado en la minimización de energía libre de Gibbs. :

$$ min \, {G(\underline{F}^\alpha, \underline{F}^\beta)} = \sum_{i=1}^c (F_i^\alpha \ln \hat{f}_i ^\alpha + F_i^\beta \ln \hat{f}_i^\beta) $$

Aquí, $F$ se refiere al número de moles y $\hat{f}$ a la fugacidad efectiva, el superíndice se refiere al índice de fase y el subíndice al índice de especie. La optimización se realiza utilizando rutinas de minimización de SciPy.

La rutina ``flash`` de Phasepy resuelve composiciones de fases para flash de dos fases a presión y temperatura constantes (PT-flash). La función ```flash``` requiere conjeturas iniciales para las composiciones de fases, los estados de las fases, es decir, ```LV``` para flash líquido-vapor o ```LL``` para flash líquido-líquido, la composición de fase global (``z``), la temperatura (``T``) y la presión (``P``) del sistema.

En este tutorial se ejemplificará el cálculo instantáneo de vapor-líquido y líquido-líquido. Los ejemplos siguientes aplican la ecuación de estado de Peng-Robinson. La regla de mezcla aplicada es Hurón Vidal Modificado combinado con UNIFAC Modificado.

Para empezar, se importan las funciones necesarias.

In [1]:
import numpy as np
from phasepy import mixture, component, preos
from phasepy.equilibrium import flash

---
### Flash para Equilibrio Líquido Vapor

Este cálculo se ejemplificará para la mezcla de etanol y agua.

In [2]:
# Inicializando la ecuación de estado y UNIFAC. 
water = component(name='water', Tc=647.13, Pc=220.55, 
                  w=0.344861, GC={'H2O':1})

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37,
                    w=0.643558, GC={'CH3':1, 'CH2':1, 'OH(P)':1})

mix = mixture(ethanol, water)

mix.unifac() 
eos = preos(mix, 'mhv_unifac')

# Condiciones de operación y concentración de entrada  
T = 350.0
P = 0.8
Z = np.array([0.4, 0.6])

# Valores iniciales para ambas fases 
x0 = np.array([0.23512692, 0.76487308])
y0 = np.array([0.5, 0.5])

# Rutina de calculo
flash(x0, y0, 'LV', Z, T, P, eos, full_output=True)

      T: 350.0
      P: 0.8
   beta: 0.46657039792253846
  error: 9.457067269309928e-09
   iter: 7
      X: array([0.23951597, 0.76048403])
     v1: 32.508879490521245
 state1: 'L'
      Y: array([0.58348128, 0.41651872])
     v2: 35869.13850298782
 state2: 'V'

### Flash para Equilibrio Líquido Vapor

Este cálculo se ejemplificará para la mezcla de agua y butanol.

In [3]:
# Inicializando la ecuación de estado y UNIFAC. 
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1})

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})
mix = water + mtbe
mix.unifac()
eos = preos(mix, 'mhv_unifac')

# Condiciones de operación y concentración de entrada  
T = 320.0
P = 1.01
Z = np.array([0.5, 0.5])

# Valores iniciales para ambas fases 
x0 = np.array([0.01, 0.99])
w0 = np.array([0.99, 0.01])

# Rutina de calculo
flash(x0, w0, 'LL', Z, T, P, eos, full_output=True)

      T: 320.0
      P: 1.01
   beta: 0.4699108915135797
  error: 2.908288133638802e-09
   iter: 7
      X: array([0.05876434, 0.94123566])
     v1: 112.09263402948574
 state1: 'L'
      Y: array([0.99774164, 0.00225836])
     v2: 21.75681728795378
 state2: 'L'

# Punto de burbuja (VLE)

El balance de masa de Rachford-Rice se reduce a las siguientes ecuaciones:

$$ FO = \sum_{i=1}^c x_i (K_i-1) = \sum_{i=1}^c y_i -1 = 0 $$

La solución de estos cálculos incluye el uso de sustitución sucesiva acelerada (ASS) para actualizar las composiciones de fases en un circuito interno y el método cuasi-Newton se usa para actualizar la presión o temperatura en un circuito externo. Si se detecta una convergencia lenta, el algoritmo intenta resolver el siguiente sistema de ecuaciones utilizando constantes de equilibrio, $K$, como variables de iteración. Esto se hace utilizando las rutinas de optimización de SciPy.

$$ f_i = \ln K_i + \ln \hat{\phi}_i^v(\underline{y}, T, P) -\ln \hat{\phi}_i^l(\underline{x}, T , P) \quad i = 1,...,c $$
$$ f_{c+1} = \sum_{i=1}^c (y_i-x_i) $$

**Nota:** estos cálculos no verifican la estabilidad de las fases.


En este tutorial se muestran ejemplos de cálculo de las propiedades del punto de burbuja utilizando la ecuación de estado de Peng-Robinson. La regla de mezcla aplicada es Hurón Vidal Modificado combinado con UNIFAC Modificado.

In [4]:
from phasepy.equilibrium import bubblePy, bubbleTy

water = component(name='water', Tc=647.13, Pc=220.55, 
                  w=0.344861, GC={'H2O':1})
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37,
                    w=0.643558, GC={'CH3':1, 'CH2':1, 'OH(P)':1})
mix = mixture(ethanol, water)

mix.unifac() 
eos = preos(mix, 'mhv_unifac')

In [5]:
### Algoritmo de punto de burbuja x, T -> y, P

# Condiciones de operación
x = np.array([0.5, 0.5])
T = 350.0

# Valores iniciales
y0 = np.array([0.5, 0.5])
P0 = 1.0

# Rutina de calculo
bubblePy(y0, P0, x, T, eos, full_output=True)

      T: 350.0
      P: 0.8969829229313713
  error: 1.1355361095866101e-12
   iter: 5
      X: array([0.5, 0.5])
     v1: 44.140674802039406
 state1: 'Liquid'
      Y: array([0.68233401, 0.31766599])
     v2: 31901.55013661451
 state2: 'Vapor'

In [6]:
### Algoritmo de punto de burbuja x, P -> y, T

# Condiciones de operación
x = np.array([0.6, 0.4])
P = 3.0

# Valores iniciales
y0 = np.array([0.5, 0.5])
T0 = 320.0

# Rutina de calculo
bubbleTy(y0, T0, x, P, eos, full_output=True)

      T: 382.84576305426117
      P: 3.0
  error: 2.3349588931748517e-10
   iter: 5
      X: array([0.6, 0.4])
     v1: 51.11127949434468
 state1: 'Liquid'
      Y: array([0.7027532, 0.2972468])
     v2: 10143.393200684526
 state2: 'Vapor'

#  Punto de Rocío (VLE)

El balance de masa de Rachford-Rice se reduce a las siguientes ecuaciones:

$$ FO = 1 - \sum_{i=1}^c \frac{y_i}{K_i} = 1 - \sum_{i=1}^c x_i = 0 $$

La solución de estos cálculos incluye el uso de sustitución sucesiva acelerada (ASS) para actualizar las composiciones de fases en un circuito interno y el método cuasi-Newton se usa para actualizar la presión o temperatura en un circuito externo. Si se detecta una convergencia lenta, el algoritmo intenta resolver el siguiente sistema de ecuaciones utilizando constantes de equilibrio, $K$, como variables de iteración. Esto se hace utilizando las rutinas de optimización de SciPy.

$$ f_i = \ln K_i + \ln \hat{\phi}_i^v(\underline{y}, T, P) -\ln \hat{\phi}_i^l(\underline{x}, T , P) \quad i = 1,...,c $$
$$ f_{c+1} = \sum_{i=1}^c (y_i-x_i) $$

**Nota:** estos cálculos no verifican la estabilidad de las fases.


En este tutorial se muestran ejemplos de cálculo de las propiedades del punto de rocío utilizando ecuación de estado de Peng-Robinson-Stryjek-Vera. La regla de mezcla aplicada es Huron Vidal modificado combinado con Redlich-Kister.

In [7]:
from phasepy import mixture, component, prsveos, rkeos
from phasepy.equilibrium import dewPx, dewTx

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    ksv=[1.27092923, 0.0440421])

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 ksv=[0.76429651, 0.04242646])

mix = mtbe + ethanol
C0 = np.array([0.02635196, -0.02855964, 0.01592515])
C1 = np.array([312.575789, 50.1476555, 5.13981131])
mix.rk(C0, C1)
eos = prsveos(mix, mixrule='mhv_rk')




In [8]:
### Algoritmo de punto de burbuja y, T -> x, P

# Condiciones de operación
y = np.array([0.5, 0.5])
T = 350.0

# Valores iniciales
x0 = np.array([0.5, 0.5])
P0 = 1.0


# Rutina de calculo
dewPx(x0, P0, y, T, eos, full_output=True)

      T: 350.0
      P: 1.5445973713541
  error: 2.96076496653086e-11
   iter: 4
      X: array([0.23512692, 0.76487308])
     v1: 79.8592556832308
 state1: 'Liquid'
      Y: array([0.5, 0.5])
     v2: 18058.942069436394
 state2: 'Vapor'

In [9]:
### Algoritmo de punto de burbuja y, P -> x, T

# Condiciones de operación
y = np.array([0.4, 0.6])
P = 1.0

# Valores iniciales
x0 = np.array([0.1, 0.9])
T0 = 360.0


# Rutina de calculo
dewTx(x0, T0, y, P, eos, full_output=True)

      T: 341.7556832715947
      P: 1.0
  error: 3.1086244689504336e-15
   iter: 5
      X: array([0.1357666, 0.8642334])
     v1: 73.24673375253408
 state1: 'Liquid'
      Y: array([0.4, 0.6])
     v2: 27627.20399825711
 state2: 'Vapor'

# Equilibrio Líquido Líquido (LLE)

La estabilidad de fase juega un papel clave durante el cálculo del equilibrio cuando se trata de más de dos fases líquidas. Para este propósito, [Gupta et al.] (https://www.sciencedirect.com/science/article/pii/037838129180021M) propusieron el siguiente balance de masas de Rachford-Rice multifásico modificado:


$$ \sum_{i=1}^c \frac{z_i (K_{ik} \exp{\theta_k}-1)}{1+ \sum\limits^{\pi}_{\substack{j=1 \\ j \neq r}}{\psi_j (K_{ij}} \exp{\theta_j} -1)} = 0 \qquad k = 1,..., \pi, k \neq r $$

Sujeto a:

$$ \psi_k \theta_k = 0 $$

En este sistema de ecuaciones, $z_i$ representa la composición global del componente $i$, $ K_{ij} = x_{ij}/x_{ir} = \hat{\phi}_{ir}/\hat{ \phi}_{ij} $ es el equilibrio constante del componente $i$ en la fase $j$ respecto a la fase de referencia $r$, y $\psi_j$ y $\theta_j$ son la fracción de fase y la variable de estabilidad de la fase $j$.

La estrategia de solución es similar al clásico flash isobárico isotérmico de dos fases. En primer lugar se debe seleccionar una fase de referencia, esta fase se considera estable durante el procedimiento. En un bucle interno, el sistema de ecuaciones se resuelve usando el método multidimensional de Newton para fracciones de fase y variables de estabilidad y luego las composiciones se actualizan en un bucle externo usando sustitución sucesiva acelerada (ASS). Una vez que el algoritmo ha convergido, la variable de estabilidad proporciona información sobre la fase. Si toma un valor de cero la fase es estable y si es positiva la fase no lo es. El método de sustitución sucesiva propuesto puede ser lento, si ese es el caso el algoritmo intenta minimizar la energía libre de Gibbs del sistema. Este procedimiento también garantiza soluciones estables y se resuelve utilizando las funciones de SciPy.

$$ min \, {G} = \sum_{k=1}^\pi \sum_{i=1}^c F_{ik} \ln \hat{f}_{ik} $$

Este ejemplo muestra la solución del equilibrio líquido-líquido usando la función ``lle``. Esta función incorpora el algoritmo descrito anteriormente. Para empezar, se importan las funciones necesarias.

Se ejemplificará el cálculo del LLE para la mezcla de agua y mtbe. Primero se configura la mezcla y sus parámetros de interacción.

In [28]:
from phasepy import component, mixture, virialgamma, unifac
from phasepy.equilibrium import lle

water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1})

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})

mix = water + mtbe
mix.unifac()
eos = virialgamma(mix, actmodel='unifac')

# Condiciones de operación
T = 320.0
P = 1.01
Z = np.array([0.5, 0.5])

# Valores iniciales
x0 = np.array([0.01, 0.99])
w0 = np.array([0.99, 0.01])

# Rutina de calculo
lle(x0, w0, Z, T, P, eos, full_output=True)

           T: 320.0
           P: 1.01
 error_outer: 2.9035040568651286e-09
 error_inner: 1.0073726451407202e-10
        iter: 7
        beta: array([0.53009228, 0.46990772])
       tetha: array([0.])
           X: array([[0.05876935, 0.94123065],
       [0.99774233, 0.00225767]])
           v: [None, None]
      states: ['L', 'L']

Adicionalmente, phasepy posee una routina para ayudar al cálculo de condiciciones iniciales, estas se pueden obtener usando la función ``lle_init``. Esto se muestra a continuación.

In [23]:
from phasepy.equilibrium import lle_init
from phasepy import preos 

eos = preos(mix, 'mhv_unifac')

# Condiciones de operación
T = 320.0
P = 1.01
z = np.array([0.5, 0.5])

# Rutina de calculo
lle_init(z, T, P, eos)

(array([9.99053445e-01, 9.46554585e-04]), array([0.25532584, 0.74467416]))

# Equilibrios sólido-líquido

En esta sección se ejemplifica los cálculos de equilibrios sólido-líquido. Para ello existen dos funciones implementadas:

- `sle`: Calcula los equilibrios Sólido-Líquido de una mezcla dada de composición global (Z) a temperatura (T) y presión (P).

- `slle`: Calcula los equilibrios Sólido-Líquido-Líquido de una mezcla dada de composición global (Z) a temperatura (T) y presión (P).

**Notas**
1. Ambas funciones utilizan un flash multifásico modificado que calcula el equilibrio y la estabilidad simultáneamente. Sin embargo, tenga en cuenta que "sle" no comprueba si la fase líquida es miscible.

2. Ambas funciones consideran las fases sólidas como fases puras.

3. La función requiere que usted configure la entalpía de fusión de los componentes y la temperatura a la que se permite solidificar.

4. Esta función ha establecido de forma predeterminada el número de fases líquidas; sin embargo, se pueden calcular múltiples fases sólidas. Asegúrese de que el número máximo de fases sea menor o igual al número de componentes de la mezcla (regla de las fases de Gibbs)

### Definición de mezcla

En este ejemplo se considerará una mezcla ternaria de ibuprofeno (1) + etanol (2) + agua (3).

Se permitirá que el ibuprofeno se solidifique. Por lo tanto, se suministran su entalpía de fusión `[J/mol]` y temperatura `[K]`.

La mezcla se modelará utilizando el modelo de coeficiente de actividad original de UNIFAC.

In [34]:
from phasepy import component, mixture, virialgamma
from phasepy.equilibrium import sle, slle

# Propiedades de cambio de fase del soluto
hf_ibuprofen = 25500 # J/mol
tf_ibuprofen = 347.15 # K

ibuprofen = component(name='ibuprofen', Tc=765., Pc=29.8, Zc=0.313, Vc=668.,
                      GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1},
                      dHf=hf_ibuprofen, Tf=tf_ibuprofen)

water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229,
                  Vc=55.9472,GC={'H2O':1})

ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168.,
                    GC={'CH3':1, 'CH2':1, 'OH':1})

mix = ibuprofen + ethanol + water
eos = virialgamma(mix, virialmodel='ideal_gas', actmodel='original_unifac')

### Equilibrios sólido-líquido

Este equilibrio de fases se calcula utilizando la función ``sle``. El coeficiente de fugacidad del líquido se obtiene como es habitual a partir del modelo del coeficiente de actividad o de una ecuación de estado cúbica. La fase sólida se considera pura y su coeficiente de fugacidad se calcula de la siguiente manera:

$$ \ln \phi^s = \ln \phi^l - \frac{\Delta H_f}{R} \left(\frac{1}{T} - \frac{1}{T_f}\right)$$ 

El equilibrio de fases se calcula con un flash que verifica el equilibrio y la estabilidad entre las fases líquida y sólida. **Nota:** No se verifica la estabilidad de la fase líquida en sí. La fase líquida aún puede ser inestable. En ese caso, se recomienda utilizar la función `slle`.

El SLE se calcula de la siguiente manera:
- `beta`: [fase líquida, fases sólidas] -> fracciones de fase.
- `tetha`: [fases sólidas] -> se supone que la fase líquida es estable. Si tetha > 0, significa que la fase es inestable.

In [36]:
#Condiciones de operación
P = 1.01325 # bar
T = 318.15 # K
Z = np.array([0.6, 0.3, 0.1])

#Rutina de calculo
sle(Z, T, P, eos, solid_phases_index=[0], full_output=True)

            T: 318.15
            P: 1.01325
  error_outer: 4.984158382631239e-11
  error_inner: 1.0168103661922023e-10
         iter: 11
         beta: array([0.64400275, 0.35599725])
        tetha: array([0.])
      X_fluid: array([[0.37888464, 0.46583652, 0.15527884]])
      v_fluid: [None]
 states_fluid: ['L']
      X_solid: array([[1., 0., 0.]])